In [ ]:
import keras, os
import tensorflow as tf
from keras.models import Model
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.layers import concatenate
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models, losses
from keras.layers import Input, Dense, Conv2D, MaxPool2D , Flatten, MaxPooling2D, GlobalAveragePooling2D ,AveragePooling2D, Dropout, Activation, BatchNormalization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Read Dataset**

In [ ]:
TRAIN_AUG_DIR = r"/content/drive/MyDrive/Train"
VALIDATION_AUG_DIR = r"/content/drive/MyDrive/Validation"

image_size=(224,224)
train_d = tf.keras.preprocessing.image_dataset_from_directory(
     TRAIN_AUG_DIR,
     labels="inferred",
     label_mode="categorical",
     color_mode="rgb",
     batch_size=64,
     image_size=image_size,
     seed=111,
     shuffle=True
    )

valid_d =tf.keras.preprocessing.image_dataset_from_directory(
    VALIDATION_AUG_DIR,
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=64,
    image_size=image_size,
    seed=111,
    shuffle=True
    )

# **History Plot**

In [ ]:
def history_plot(history):
  fig, axs = plt.subplots(2, 1, figsize=(15,15))
  axs[0].plot(history.history['loss'])
  axs[0].plot(history.history['val_loss'])
  axs[0].title.set_text('Training Loss vs Validation Loss')
  axs[0].set_xlabel('Epochs')
  axs[0].set_ylabel('Loss')
  axs[0].legend(['Train', 'Val'])

  axs[1].plot(history.history['accuracy'])
  axs[1].plot(history.history['val_accuracy'])
  axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
  axs[1].set_xlabel('Epochs')
  axs[1].set_ylabel('Accuracy')
  axs[1].legend(['Train', 'Val'])

# **Inception Model**

In [ ]:
filters_1x1=64
filters_3x3_reduce=96
filters_3x3=128
filters_5x5_reduce=16
filters_5x5=32
filters_pool=32

input_img = Input(shape = in_shape)

conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)

conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)

pool = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
pool = Conv2D(filters_pool, (1, 1), padding='same', activation='relu')(pool)

output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool], axis=3)

output = Flatten()(output)
out    = Dense(num_classes, activation='softmax')(output)
model = Model(inputs = input_img, outputs = out)

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)

In [ ]:
history_plot(history)

# **Save Model**

In [ ]:
# from keras.models import model_from_json
# import os

# model_json = model.to_json()

# with open("model.json", "w") as json_file:
#     json_file.write(model_json)

# model.save_weights(os.path.join(os.getcwd(), 'model.h5'))